In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "seaborn"

In [2]:
import sys
sys.path.append("..")
from pygalaxy import Euler_symplectic, ADB6
from pygalaxy.barnes_hut_array import compute_energy
#from pygalaxy.naive import compute_energy

In [3]:
def init_solar_system():
    
    bodies = np.array([[        0, 0, 0,      0], #sun
                       [    -46e9, 0, 0, -58980], #mercury
                       [ -10748e7, 0, 0, -35260], #venus
                       [-147095e6, 0, 0, -30300], #earth
                       [ -20662e7, 0, 0, -26500], #mars
                       #[ -74052e7, 0, 0, -13720], #jupiter
                       #[-135255e7, 0, 0, -10180], #saturn
                       #[ -27413e8, 0, 0,  -7110], #uranus
                       #[-444445e7, 0, 0,  -5500], #neptune
                      ])

    mass = np.array([   1.989e30, # sun
                      0.33011e24, #mercury
                       4.8675e24, #venus
                        5.972e24, #earth
                       6.4171e23, #mars
                      #1898.19e24, #jupiter
                      # 568.34e24, #saturn
                      # 86.813e24, #uranus
                      #102.413e24, #neptune 
                     ])

    return mass, bodies

In [4]:
dt = 86400  # 1 day in seconds

mass, particles = init_solar_system()

nbodies = particles.shape[0]
##print("nb bodies =", nbodies)

nt = 1000

time_method = Euler_symplectic(dt, nbodies, compute_energy)
#time_method = ADB6(dt, nbodies, compute_energy)

coords = np.zeros((nt, *particles[:, :2].shape))
coords[0] = particles[:, :2]

velocities = np.zeros((nt, *particles[:, 2:].shape))
velocities[0] = particles[:, 2:]

for i in range(nt-1):
    time_method.update(mass, particles)
    coords[i+1] = particles[:, :2]
    velocities[i+1] = particles[:, 2:]

fig = go.Figure()


planets = ['sun', 'mercury', 'venus', 'earth', 'mars', 'jupiter', 
           'saturn', 'uranus', 'neptune']

for i in range(5):
    if i==0: mode='markers'
    else: mode='lines'
    fig.add_trace(go.Scattergl(x=coords[:,i,0], y=coords[:,i,1], mode=mode, name=planets[i]))
    #fig.add_trace(go.Scattergl(x=[coords[-1,i,0]], y=[coords[-1,i,1]], mode='markers', showlegend=False, marker_color='grey'))


fig.update_layout(title='Solar system')
fig.update_xaxes(title='x', exponentformat='e')
fig.update_yaxes(title='y', exponentformat='e')
fig.show()